In [ ]:
# Install core dependencies
!pip install -q numpy scipy matplotlib mpmath

# Optional: if your script uses CLI arguments and file access
!pip install -q argparse

# Optional: for faster file loading or JSON operations (usually pre-installed)
!pip install -q orjson

# GPU-accelerated FFTs and array ops (for CuPy port version)
# Comment this out if using CPU-only mpmath/NumPy version
#!pip install -q cupy-cuda11x  # or `cupy-cuda12x` for CUDA 12 Colab runtimes

# Make output directory
!mkdir -p /content/output


In [ ]:
%%writefile proof_colab.py
import os
try:
    import cupy as cp
    from cupy.fft import fftn, ifftn
    gpu_mode = True
    print("Using CuPy (GPU) for arrays and FFTs")
except ImportError:
    import numpy as cp
    from numpy.fft import fftn, ifftn
    gpu_mode = False
    print("CuPy not found! Using CPU fallback; performance will be slow.")

import numpy as np
import argparse, sys, json
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import hashlib

# --- Interval arithmetic emulation: (mid, width) floats --- #
def interval(x, w=0.0):
    return (float(x), float(w))

def ia_add(a, b):
    return (a[0] + b[0], a[1] + b[1])

def ia_sub(a, b):
    return (a[0] - b[0], a[1] + b[1])

def ia_mul(a, b):
    m = a[0] * b[0]
    r = abs(a[0]) * b[1] + abs(b[0]) * a[1] + a[1] * b[1]
    return (m, r)

def ia_div(a, b):
    if b[0] == 0:
        return (0, 1e16)
    m = a[0]/b[0]
    r = abs(m)*(a[1]/abs(a[0] + 1e-20) + b[1]/abs(b[0] + 1e-20))
    return (m, r)

def ia_pow(a, expo):
    m = a[0]**expo
    if abs(a[0]) > 1e-10:
        r = abs(expo)*abs(a[0])**(expo - 1)*a[1]
    else:
        r = a[1]
    return (m, r)

def ia_abs(a):
    return (abs(a[0]), a[1])

def ia_max(arr):
    if len(arr) == 0:
        return (0, 0)
    m = max([x[0] for x in arr])
    r = max([x[1] for x in arr])
    return (m, r)

def ia_sum(arr):
    if len(arr) == 0:
        return (0, 0)
    m = sum([x[0] for x in arr])
    r = sum([x[1] for x in arr])
    return (m, r)

def ia_mean(arr):
    if len(arr) == 0:
        return (0, 0)
    m = sum([x[0] for x in arr]) / len(arr)
    r = sum([x[1] for x in arr]) / len(arr)
    return (m, r)

def interval_repr(a):
    return f"{a[0]:.6g} ± {a[1]:.2e}"

# --- Dyadic filter FFT tools --- #
def fftfreq(N):
    return cp.fft.fftfreq(N) * N

def dyadic_filter(N, j):
    kx, ky, kz = [fftfreq(N)]*3
    Kx, Ky, Kz = cp.meshgrid(kx, ky, kz, indexing='ij')
    k_mag = cp.sqrt(Kx**2 + Ky**2 + Kz**2)
    low, high = 2**j, 2**(j+1)
    mask = (k_mag >= low) & (k_mag < high)
    return mask.astype(cp.float32)

# --- Field generation --- #
def make_vortex_tube(N, amplitude=1.0, core_radius=None):
    X, Y, Z = cp.meshgrid(cp.arange(N), cp.arange(N), cp.arange(N), indexing='ij')
    xc, yc = N/2, N/2
    r = cp.sqrt((X - xc)**2 + (Y - yc)**2)
    if core_radius is None:
        core_radius = N/16
    omega = cp.zeros((3, N, N, N), dtype=cp.float32)
    tube = amplitude * cp.exp(-r**2 / (2*core_radius**2))
    phi = cp.arctan2(Y - yc, X - xc)
    # swirl
    omega[0] = -tube * cp.sin(phi)
    omega[1] = tube * cp.cos(phi)
    return omega

def make_shell_localized(N, j, amplitude=1.0, seed=0):
    rng = np.random.default_rng(seed)
    fft_field = cp.zeros((3, N, N, N), dtype=cp.complex64)
    mask = dyadic_filter(N, j)
    for c in range(3):
        random_phase = cp.asarray(np.exp(1j * 2*np.pi * rng.random((N,N,N))))
        random_mag = cp.asarray(rng.normal(0,1,(N,N,N)))
        fft_field[c] = mask * amplitude * random_phase * random_mag

    kx = fftfreq(N)
    ky = fftfreq(N)
    kz = fftfreq(N)
    Kx, Ky, Kz = cp.meshgrid(kx, ky, kz, indexing='ij')
    K = cp.array([Kx, Ky, Kz])

    # Project out divergence
    for idx in cp.ndindex((N, N, N)):
        kvec = K[:, idx[0], idx[1], idx[2]]
        if cp.linalg.norm(kvec) < 1e-8:
            continue
        omega_here = fft_field[:, idx[0], idx[1], idx[2]]
        proj = cp.dot(omega_here, kvec) / (cp.dot(kvec, kvec) + 1e-15)
        fft_field[:, idx[0], idx[1], idx[2]] -= proj * kvec

    # channel-by-channel inverse FFT
    field = cp.zeros((3, N, N, N), dtype=cp.float32)
    for c in range(3):
        field[c] = cp.real(ifftn(fft_field[c], axes=(0,1,2)))
    return field

def make_boundary_layer(N, amplitude=1.0, thickness=None):
    if thickness is None:
        thickness = N/16
    X = cp.arange(N).reshape(N,1,1)
    profile = amplitude * cp.exp(-((X - N//8)**2) / (2*thickness**2))
    omega = cp.zeros((3,N,N,N), dtype=cp.float32)
    omega[2] = cp.tile(profile, (1,N,N))
    return omega

# --- Chunked / channel-wise transforms to reduce memory usage --- #
def chunked_fftn(data_4d):
    """
    Perform an FFT channel by channel to reduce peak memory usage.
    data_4d shape: (C, Nx, Nx, Nx)
    returns same shape, but complex
    """
    C = data_4d.shape[0]
    out = cp.zeros_like(data_4d, dtype=cp.complex64)
    for c in range(C):
        out[c] = fftn(data_4d[c], axes=(0,1,2))
    return out

def chunked_ifftn(data_4d):
    """
    Perform an inverse FFT channel by channel to reduce peak memory usage.
    data_4d shape: (C, Nx, Nx, Nx)
    returns same shape, but real
    """
    C = data_4d.shape[0]
    out = cp.zeros((C,) + data_4d.shape[1:], dtype=cp.float32)
    for c in range(C):
        out[c] = cp.real(ifftn(data_4d[c], axes=(0,1,2)))
    return out

# --- Biot-Savart and derivatives --- #
def biot_savart(omega, verbose=True):
    N = omega.shape[1]
    omega_hat = chunked_fftn(omega)

    kx, ky, kz = [fftfreq(N)]*3
    Kx, Ky, Kz = cp.meshgrid(kx, ky, kz, indexing='ij')
    k2 = Kx**2 + Ky**2 + Kz**2 + 1e-10
    u_hat = cp.zeros_like(omega_hat, dtype=cp.complex64)

    # cross-product in freq domain
    u_hat[0] = (1j * (Ky*omega_hat[2] - Kz*omega_hat[1])) / k2
    u_hat[1] = (1j * (Kz*omega_hat[0] - Kx*omega_hat[2])) / k2
    u_hat[2] = (1j * (Kx*omega_hat[1] - Ky*omega_hat[0])) / k2

    u = chunked_ifftn(u_hat)

    # check divergence
    divu_hat = (
        1j * Kx * chunked_fftn(u[0:1])[0] +
        1j * Ky * chunked_fftn(u[1:2])[0] +
        1j * Kz * chunked_fftn(u[2:3])[0]
    )
    divu = cp.real(ifftn(divu_hat, axes=(0,1,2)))
    maxdiv = float(cp.abs(divu).max())
    if verbose:
        print(f"[biot_savart] max|div u| = {maxdiv:.2e}")

    return u

def compute_gradients(u):
    # shape u = (3, N, N, N)
    N = u.shape[1]
    u_hat = chunked_fftn(u)
    kx, ky, kz = [fftfreq(N)]*3
    Kx, Ky, Kz = cp.meshgrid(kx, ky, kz, indexing='ij')
    K = cp.array([Kx, Ky, Kz])

    grad_u = cp.zeros((3,3,N,N,N), dtype=cp.float32)
    for alpha in range(3):
        for beta in range(3):
            tmp = 1j * K[beta] * u_hat[alpha]
            grad_comp = cp.real(ifftn(tmp, axes=(0,1,2)))
            grad_u[alpha, beta] = grad_comp.astype(cp.float32)
    return grad_u

def compute_strain(grad_u):
    # grad_u shape = (3,3,N,N,N)
    return 0.5 * (grad_u + cp.transpose(grad_u, (1,0,2,3,4)))

# --- Batched principal strain computation --- #
def principal_strain_evectors(S, batch_size=16384):
    # S shape = (3,3,N,N,N), symmetrical
    N = S.shape[2]
    S_batched = S.transpose(2,3,4,0,1).reshape(-1,3,3).astype(cp.float32)
    # ensure symmetrical
    S_batched = 0.5 * (S_batched + cp.transpose(S_batched, (0,2,1)))
    total = S_batched.shape[0]
    out = cp.empty((total, 3), dtype=cp.float32)
    for start in range(0, total, batch_size):
        stop = min(start + batch_size, total)
        sb = S_batched[start:stop]
        if cp.all(sb == 0):
            out[start:stop, :] = 0.
            out[start:stop, 0] = 1.
            continue
        try:
            vals, vecs = cp.linalg.eigh(sb)
            pv = vecs[:,:,-1]
            pv_norm = cp.linalg.norm(pv, axis=1, keepdims=True)
            pv = pv / (pv_norm + 1e-30)
            out[start:stop, :] = pv
        except Exception as e:
            print(f"Eigen failure on batch {start}:{stop}: {e}")
            out[start:stop, :] = 0.
            out[start:stop, 0] = 1.
    e1 = out.T.reshape(3, N, N, N)
    return e1

# --- Filter with chunked FFT to reduce memory usage --- #
def apply_dyadic_filter(vort, mask):
    # vort shape: (3,N,N,N); mask shape: (N,N,N)
    out = cp.zeros_like(vort)
    for c in range(3):
        v_hat = fftn(vort[c], axes=(0,1,2))
        out_hat = v_hat * mask
        out[c] = cp.real(ifftn(out_hat, axes=(0,1,2)))
        del v_hat, out_hat
        cp.get_default_memory_pool().free_all_blocks()
    return out

def linf_norm(arr):
    return float(cp.abs(arr).max())

def shell_norm_interval(omega_j):
    v = cp.abs(omega_j)
    maxval = float(v.max())
    esterr = 1e-7 * maxval
    return (maxval, esterr)

def vorticity_rhs(omega, nu):
    u = biot_savart(omega, verbose=False)
    grad_u = compute_gradients(u)
    stretch = cp.zeros_like(omega)
    for alpha in range(3):
        for beta in range(3):
            stretch[alpha] += omega[beta] * grad_u[alpha,beta]

    # diffusion
    omega_hat = chunked_fftn(omega)
    N = omega.shape[1]
    kx, ky, kz = [fftfreq(N)]*3
    Kx, Ky, Kz = cp.meshgrid(kx, ky, kz, indexing='ij')
    k2 = Kx**2 + Ky**2 + Kz**2
    laplacian = cp.zeros_like(omega)
    for c in range(3):
        laplacian_hat = -k2 * omega_hat[c]
        laplacian[c] = cp.real(ifftn(laplacian_hat, axes=(0,1,2)))
        del laplacian_hat
        cp.get_default_memory_pool().free_all_blocks()

    return nu*laplacian + stretch

def time_evolve(omega, nu, dt):
    return omega + dt * vorticity_rhs(omega, nu)

# ---- Main compute function --- #
def main_compute(omega_t, omega_tpdt, grad_u, alpha, nu,
                 j_min, j_max, dt, eps, plot=False, validate=False, context='mainrun'):
    N = omega_t.shape[1]
    shells = list(range(j_min, j_max + 1))
    per_shell = {}
    norm_mids, norm_wds, align_mids, align_wds = [], [], [], []

    if grad_u is None:
        print("  (Computing velocity and grad_u from vorticity via Biot-Savart)")
        u = biot_savart(omega_t)
        grad_u = compute_gradients(u)

    S = compute_strain(grad_u)
    e1 = principal_strain_evectors(S)

    for j in shells:
        mask = dyadic_filter(N, j)
        omega_j = apply_dyadic_filter(omega_t, mask)
        if cp.all(cp.abs(omega_j) < 1e-12):
            norm_j = (0.0, 0.0)
            align_j = (0.0, 0.0)
        else:
            norms = cp.sqrt(cp.sum(omega_j**2, axis=0))
            maxval = float(norms.max())
            esterr = 1e-7 * maxval
            norm_j = (maxval, esterr)
            norms_flat = norms.ravel()
            w_dot_e_flat = cp.sum(omega_j*e1, axis=0).ravel()
            selector = (norms_flat > eps)
            if cp.any(selector):
                num = w_dot_e_flat[selector]
                denom = norms_flat[selector] + 1e-30
                ratios = cp.abs(num / denom)
                ratios_np = cp.asnumpy(ratios)
                aligns = [(float(r), 1e-7*float(r)) for r in ratios_np]
                align_j = ia_mean(aligns)
            else:
                align_j = (0.0, 0.0)
        per_shell[j] = {'norm': norm_j, 'align': align_j}
        norm_mids.append(norm_j[0])
        norm_wds.append(norm_j[1])
        align_mids.append(align_j[0])
        align_wds.append(align_j[1])

    Y = ia_sum([
        ia_mul((2**(alpha*j),0), s['norm'])
        for j,s in per_shell.items()
    ])
    align_sup = ia_max([s['align'] for s in per_shell.values()])

    if omega_tpdt is not None:
        per_shell_p = []
        for j in shells:
            mask = dyadic_filter(N, j)
            omega_j_p = apply_dyadic_filter(omega_tpdt, mask)
            if cp.all(cp.abs(omega_j_p) < 1e-12):
                norm_j_p = (0.0, 0.0)
            else:
                norm_j_p = shell_norm_interval(omega_j_p)
            per_shell_p.append(ia_mul((2**(alpha*j),0), norm_j_p))
        Y_p = ia_sum(per_shell_p)
        dYdt = ia_div(ia_sub(Y_p, Y), (dt,0))
    else:
        dYdt = None

    delta = 2.0 / alpha
    Lambda = 2.0**alpha
    C_B, C_P, C_C, C_R, C_overlap = 32.0, 2.0, 4.0, 1.73205, 3.0
    C_nl = C_B * C_P * C_C * C_R * C_overlap

    if dYdt is not None:
        Ysq = ia_mul(Y, Y)
        Ypow = ia_pow(Y, 1 + delta)
        rhs1 = ia_mul((C_nl,0), ia_mul(Ysq, align_sup))
        rhs2 = ia_mul((nu * Lambda,0), Ypow)
        rhs = ia_sub(rhs1, rhs2)
        holds = dYdt[0] + dYdt[1] <= rhs[0] - rhs[1]
    else:
        rhs = None
        holds = None

    diagnostics = {
        'context': context,
        'shells': [
            {'j': int(j),
             'norm': interval_repr(per_shell[j]['norm']),
             'align': interval_repr(per_shell[j]['align'])}
            for j in shells
        ],
        'Y': interval_repr(Y),
        'align_sup': interval_repr(align_sup),
        'delta': delta,
        'Lambda': Lambda,
        'C_nl': C_nl,
        'dYdt': interval_repr(dYdt) if dYdt is not None else "",
        'RHS': interval_repr(rhs) if rhs is not None else "",
        'holds': holds,
        '_entropic_per_shell': [
            dict(j=int(j), norm=per_shell[j]['norm'])
            for j in shells
        ]
    }

    print(f"\nRecursive norm Y(t):          {diagnostics['Y']}")
    print(f"sup_j alignment:              {diagnostics['align_sup']}")
    if dYdt is not None:
        print(f"dY/dt:                        {diagnostics['dYdt']}")
        print(f"RHS:                          {diagnostics['RHS']}")
        if holds:
            print("[Inequality verified]: This timestep is within bounds.\n")
        else:
            print("!!! WARNING: Inequality violated (with certified intervals) !!!\n")
    else:
        print("dY/dt not available (only single time). No inequality check.")

    if plot:
        fig, axs = plt.subplots(2,1,figsize=(7,6), sharex=True)
        axs[0].bar(shells, norm_mids, yerr=norm_wds, capsize=5)
        axs[0].set_ylabel(r"$\|\Delta_j\omega\|_{L^\infty}$")
        axs[1].bar(shells, align_mids, yerr=align_wds, capsize=5)
        axs[1].set_ylabel(r"$\mathcal{A}_j$")
        axs[1].set_xlabel('Shell $j$')
        plt.suptitle("Per-shell norms and alignments")
        plt.tight_layout()
        fig.savefig(f"/content/output/per_shell_{context}.png")

        if dYdt is not None:
            plt.figure()
            plt.bar(["dY/dt","RHS"], [dYdt[0], rhs[0]], yerr=[dYdt[1], rhs[1]], capsize=8)
            plt.title("Recursive damping inequality: LHS vs RHS\n(mid ± width)")
            plt.savefig(f"/content/output/inequality_{context}.png")

    with open(f"/content/output/diagnostics_log_{context}.json",'w', encoding="utf-8") as f:
        json.dump(diagnostics, f, indent=2)

    return {
        'holds': holds,
        'dYdt': dYdt,
        'rhs': rhs,
        'Y': Y,
        'align_sup': align_sup,
        'delta': delta,
        'Lambda': Lambda,
        'C_nl': C_nl,
        '_entropic_per_shell': [
            dict(j=int(j), norm=per_shell[j]['norm'])
            for j in shells
        ]
    }, diagnostics

def export_certificate(fname, dYdt, rhs, C_nl, delta, Lambda, Y, align_sup):
    lines = []
    lines.append("-- Certified Clay-Companion Recursive Inequality Certificate --")
    lines.append(f"-- C_nl = {C_nl}")
    lines.append(f"-- delta = {delta}")
    lines.append(f"-- Lambda = {Lambda}")
    lines.append(f"-- Y = {Y}")
    lines.append(f"-- align_sup = {align_sup}")
    lines.append(f"-- Computed (LHS = dY/dt): {dYdt[0]} ± {dYdt[1]}")
    lines.append(f"-- Computed (RHS):          {rhs[0]} ± {rhs[1]}")
    lines.append("theorem verified_t0 : dYdt ≤ RHS := by float_solver")
    # Note: Use encoding="utf-8" to avoid Unicode issues with "±".
    with open(fname, "w", encoding="utf-8") as f:
        for L in lines:
            f.write(L + '\n')
    print(f"Wrote Lean-style proof certificate to {fname}")

def validate_damping_terms(Y, dYdt, rhs, align_sup, delta, Lambda, C_nl):
    assert isinstance(Y, tuple) and isinstance(dYdt, tuple) and isinstance(rhs, tuple), \
        "Y, dY/dt, and rhs must be interval tuples"
    assert align_sup[0] <= 1.0 + 1e-6, f"Alignment factor exceeds 1 (align_sup={align_sup[0]})"
    assert Lambda > 1.0, f"Lambda must be > 1 (Lambda={Lambda})"
    assert delta > 0.0, f"delta must be positive (delta={delta})"
    assert C_nl > 0.0, f"C_nl must be positive (C_nl={C_nl})"
    assert rhs[0] >= 0 or abs(rhs[0]) < 1e-5, f"RHS is negative ({rhs[0]:.2e}), implies anti-damping"
    # We expect dY/dt to be non-positive in a damping scenario
    assert dYdt[0] < 0, f"dY/dt is positive ({dYdt[0]:.2e}) - unexpected growth"
    return True

def compute_entropy(shells):
    H = 0.0
    for s in shells:
        if isinstance(s['norm'], str):
            # Convert from "a ± b" format if encountered
            parts = s['norm'].split('±')
            mid = float(parts[0].strip())
        else:
            mid = s['norm'][0]
        if mid > 0:
            H += mid * np.log(mid)
    return H

def save_forensic_failure(timestep, omega, Y, dYdt, rhs, align_sup, folder="/content/output"):
    os.makedirs(folder, exist_ok=True)
    hash_key = hashlib.md5(str((Y, dYdt, rhs, align_sup)).encode()).hexdigest()[:12]
    try:
        cp.save(f"{folder}/counterexample_t{timestep:03d}.npy", omega)
    except Exception:
        np.save(f"{folder}/counterexample_t{timestep:03d}.npy", cp.asnumpy(omega))
    with open(f"{folder}/failure_log_t{timestep:03d}.txt", "w", encoding="utf-8") as f:
        f.write(f"FAILURE HASH: {hash_key}\n")
        f.write(f"Y(t): {Y}\n")
        f.write(f"dY/dt: {dYdt}\n")
        f.write(f"RHS: {rhs}\n")
        f.write(f"align_sup: {align_sup}\n")

def float_sanity_check(intervals):
    warnings = []
    for name, val in intervals.items():
        if val is None:
            continue
        if abs(val[0]) > 1e-12 and val[1]/abs(val[0]) > 1e-2:
            warnings.append(f"⚠️ Interval {name} is unstable: {val[0]:.5g} ± {val[1]:.2e}")
    return warnings

# --- Additional test routines --- #
def stress_commutator(N=64, alpha=2.5, j_min=2, j_max=6,
                      dt=1e-4, timesteps=1, strict=False,
                      plot=False, run_validation_checks=False,
                      nu=0.01, export_cert=False, eps=1e-10):
    """
    Commutator test: Injects j=2 and j=5 shells to check nonlinear 'leakage' between scales.
    Demonstrates a scenario for short-time stepping with multiple shells.
    """
    print("[commutator test] j_min={}, j_max={}, timesteps={}".format(j_min, j_max, timesteps))
    omega = make_shell_localized(N, j=2) + make_shell_localized(N, j=5)
    for t in range(timesteps):
        context = f"commutator_test_t{t}"
        omega_next = time_evolve(omega, nu, dt)
        # Usually we do the same analysis as main_compute:
        res, diagnostics = main_compute(
            omega, omega_next,
            grad_u=None,
            alpha=alpha,
            nu=nu,
            j_min=j_min,
            j_max=j_max,
            dt=dt,
            eps=eps,
            plot=plot,
            validate=run_validation_checks,
            context=context
        )

        if run_validation_checks:
            try:
                validate_damping_terms(
                    res['Y'], res['dYdt'], res['rhs'],
                    res['align_sup'], res['delta'],
                    res['Lambda'], res['C_nl']
                )
            except AssertionError as e:
                print(f"[VALIDATION ERROR] {e}")
                save_forensic_failure(
                    timestep=t,
                    omega=omega,
                    Y=res['Y'],
                    dYdt=res['dYdt'],
                    rhs=res['rhs'],
                    align_sup=res['align_sup']
                )
                if strict:
                    sys.exit(1)

            w = float_sanity_check({
                'Y': res['Y'],
                'dY/dt': res['dYdt'],
                'RHS': res['rhs'],
                'align_sup': res['align_sup']
            })
            for ww in w:
                print(ww)

        if export_cert and (res['dYdt'] is not None) and (res['rhs'] is not None):
            export_certificate(
                f"/content/output/certificate_{context}.lean",
                res['dYdt'],
                res['rhs'],
                res['C_nl'],
                res['delta'],
                res['Lambda'],
                res['Y'],
                res['align_sup']
            )

        if strict and res['holds'] is False:
            print("FAIL: Damping inequality violated in commutator test at step", t)
            save_forensic_failure(
                timestep=t,
                omega=omega,
                Y=res['Y'],
                dYdt=res['dYdt'],
                rhs=res['rhs'],
                align_sup=res['align_sup']
            )
            sys.exit(1)

        omega = omega_next.copy()

    print("[commutator test] Completed.")

def simulate_blowup(N=64, alpha=2.5, j_min=1, j_max=5,
                    dt=1e-4, timesteps=1, strict=False,
                    plot=False, run_validation_checks=False,
                    nu=0.01, export_cert=False, eps=1e-10):
    """
    Blowup test: Force alignment near 1 to simulate a worst-case growth scenario.
    """
    print("[blowup test] j_min={}, j_max={}, timesteps={}".format(j_min, j_max, timesteps))
    # Make a strong vortex tube, then forcibly align it
    omega = make_vortex_tube(N) * 10.0
    e1 = cp.ones_like(omega)
    # Force full alignment
    norms = cp.sqrt(cp.sum(omega**2, axis=0, keepdims=True))
    omega = e1 * norms

    for t in range(timesteps):
        context = f"blowup_test_t{t}"
        omega_next = time_evolve(omega, nu, dt)
        res, diagnostics = main_compute(
            omega, omega_next,
            grad_u=None,
            alpha=alpha,
            nu=nu,
            j_min=j_min,
            j_max=j_max,
            dt=dt,
            eps=eps,
            plot=plot,
            validate=run_validation_checks,
            context=context
        )

        if run_validation_checks:
            try:
                validate_damping_terms(
                    res['Y'], res['dYdt'], res['rhs'],
                    res['align_sup'], res['delta'],
                    res['Lambda'], res['C_nl']
                )
            except AssertionError as e:
                print(f"[VALIDATION ERROR] {e}")
                save_forensic_failure(
                    timestep=t,
                    omega=omega,
                    Y=res['Y'],
                    dYdt=res['dYdt'],
                    rhs=res['rhs'],
                    align_sup=res['align_sup']
                )
                if strict:
                    sys.exit(1)

            w = float_sanity_check({
                'Y': res['Y'],
                'dY/dt': res['dYdt'],
                'RHS': res['rhs'],
                'align_sup': res['align_sup']
            })
            for ww in w:
                print(ww)

        if export_cert and (res['dYdt'] is not None) and (res['rhs'] is not None):
            export_certificate(
                f"/content/output/certificate_{context}.lean",
                res['dYdt'],
                res['rhs'],
                res['C_nl'],
                res['delta'],
                res['Lambda'],
                res['Y'],
                res['align_sup']
            )

        if strict and res['holds'] is False:
            print("FAIL: Damping inequality violated in blowup test at step", t)
            save_forensic_failure(
                timestep=t,
                omega=omega,
                Y=res['Y'],
                dYdt=res['dYdt'],
                rhs=res['rhs'],
                align_sup=res['align_sup']
            )
            sys.exit(1)

        omega = omega_next.copy()

    print("[blowup test] Completed.")

def time_reverse_test(N=64, alpha=2.5, j_min=2, j_max=5,
                      dt=1e-4, timesteps=1, strict=False,
                      plot=False, run_validation_checks=False,
                      nu=0.01, export_cert=False, eps=1e-10):
    """
    Time reversal test: forward and backward time evolution to check
    reversibility of the solver.
    """
    print("[time reversal test] j_min={}, j_max={}, timesteps={}".format(j_min, j_max, timesteps))
    omega_t = make_vortex_tube(N)
    # We only do a single forward/back step to test reversibility (or repeat if timesteps>1).
    for t in range(timesteps):
        omega_tpdt = time_evolve(omega_t, nu, dt)
        omega_back = time_evolve(omega_tpdt, nu, -dt)
        err = float(cp.abs(omega_back - omega_t).max())
        print(f"[time reverse] Step={t}, forward->back error = {err:.3e}")

        # We can still run main_compute if we like, but typically this test is about reversibility.
        res, diagnostics = main_compute(
            omega_t, omega_tpdt,
            grad_u=None,
            alpha=alpha,
            nu=nu,
            j_min=j_min,
            j_max=j_max,
            dt=dt,
            eps=eps,
            plot=plot,
            validate=run_validation_checks,
            context=f"time_reverse_t{t}"
        )

        if run_validation_checks:
            try:
                validate_damping_terms(
                    res['Y'], res['dYdt'], res['rhs'],
                    res['align_sup'], res['delta'],
                    res['Lambda'], res['C_nl']
                )
            except AssertionError as e:
                print(f"[VALIDATION ERROR] {e}")
                save_forensic_failure(
                    timestep=t,
                    omega=omega_t,
                    Y=res['Y'],
                    dYdt=res['dYdt'],
                    rhs=res['rhs'],
                    align_sup=res['align_sup']
                )
                if strict:
                    sys.exit(1)

            w = float_sanity_check({
                'Y': res['Y'],
                'dY/dt': res['dYdt'],
                'RHS': res['rhs'],
                'align_sup': res['align_sup']
            })
            for ww in w:
                print(ww)

        if export_cert and (res['dYdt'] is not None) and (res['rhs'] is not None):
            export_certificate(
                f"/content/output/certificate_time_reverse_t{t}.lean",
                res['dYdt'],
                res['rhs'],
                res['C_nl'],
                res['delta'],
                res['Lambda'],
                res['Y'],
                res['align_sup']
            )

        if strict and res['holds'] is False:
            print("FAIL: Damping inequality violated in time_reverse test at step", t)
            save_forensic_failure(
                timestep=t,
                omega=omega_t,
                Y=res['Y'],
                dYdt=res['dYdt'],
                rhs=res['rhs'],
                align_sup=res['align_sup']
            )
            sys.exit(1)

    print("[time reverse test] Completed.")


def main():
    parser = argparse.ArgumentParser(
        description="Recursive damping proof + additional tests."
    )

    # Base / shared arguments
    parser.add_argument('--N', type=int, default=512,
                        help="Grid size (default 512).")
    parser.add_argument('--alpha', type=float, default=2.5,
                        help="Scaling exponent alpha (default 2.5).")
    parser.add_argument('--nu', type=float, default=0.01,
                        help="Viscosity coefficient (default 0.01).")
    parser.add_argument('--eps', type=float, default=1e-10,
                        help="Threshold epsilon for ignoring near-zero shells (default 1e-10).")
    parser.add_argument('--plot', action='store_true',
                        help="Enable plotting per-shell metrics.")
    parser.add_argument('--export_cert', action='store_true',
                        help="Export Lean-style proof certificates.")
    parser.add_argument('--run_validation_checks', action='store_true',
                        help="Perform extra validation checks on intervals.")
    parser.add_argument('--input', type=str,
                        help="Path to omega_t.npy for the initial vorticity field.")
    parser.add_argument('--input2', type=str,
                        help="Path to omega_tpdt.npy for the next-timestep vorticity field.")
    parser.add_argument('--grad_u', type=str,
                        help="Optional path to grad_u.npy if precomputed.")

    # Additional flags for separate tests
    parser.add_argument('--run_blowup_test', action='store_true',
                        help="Run the blowup test scenario after main run.")
    parser.add_argument('--run_commutator_test', action='store_true',
                        help="Run the commutator test scenario after main run.")
    parser.add_argument('--run_time_reverse_test', action='store_true',
                        help="Run the time-reverse test scenario after main run.")

    # Test/evolution arguments
    parser.add_argument('--j_min', type=int, default=6,
                        help="Minimum exponent j for the dyadic shell range (default 6).")
    parser.add_argument('--j_max', type=int, default=9,
                        help="Maximum exponent j for the dyadic shell range (default 9).")
    parser.add_argument('--dt', type=float, default=1e-4,
                        help="Time step for vorticity evolution (default 1e-4).")
    parser.add_argument('--timesteps', type=int, default=0,
                        help="Number of time steps to run and certify (default 0).")
    parser.add_argument('--strict', action='store_true',
                        help="Exit with error if any violation occurs in the inequality check.")

    args = parser.parse_args()

    # Collect test flags to run later
    run_blowup = args.run_blowup_test
    run_commutator = args.run_commutator_test
    run_time_reverse = args.run_time_reverse_test

    # 1) Load or generate initial field
    if args.input is not None:
        omega_t = cp.load(args.input)
        print(f"Loaded omega_t from {args.input}, shape {omega_t.shape}.")
    else:
        print("No --input provided. Generating a synthetic 'vortex tube' field.")
        omega_t = make_vortex_tube(args.N)

    # 2) Optionally load next-timestep field
    omega_tpdt = None
    if args.input2 is not None:
        omega_tpdt = cp.load(args.input2)
        print(f"Loaded omega_tpdt from {args.input2}, shape {omega_tpdt.shape}.")

    # 3) Optionally load grad_u if provided
    grad_u = None
    if args.grad_u is not None and os.path.isfile(args.grad_u):
        grad_u = cp.load(args.grad_u)
        print(f"Loaded grad_u from {args.grad_u}, shape {grad_u.shape}.")

    # 4) Multi-step evolution if timesteps > 0
    if args.timesteps > 0:
        print(f"Running {args.timesteps} time steps with dt={args.dt} and verifying at each step.")
        omega = omega_t.copy()
        for t in range(args.timesteps):
            context = f"timestep_{t:03d}"
            omega_next = time_evolve(omega, args.nu, args.dt)
            res, diagnostics = main_compute(
                omega, omega_next,
                grad_u,
                args.alpha,
                args.nu,
                j_min=args.j_min,
                j_max=args.j_max,
                dt=args.dt,
                eps=args.eps,
                plot=args.plot,
                validate=args.run_validation_checks,
                context=context
            )

            if args.run_validation_checks:
                try:
                    validate_damping_terms(
                        res['Y'], res['dYdt'], res['rhs'],
                        res['align_sup'], res['delta'],
                        res['Lambda'], res['C_nl']
                    )
                except AssertionError as e:
                    print(f"[VALIDATION ERROR] {e}")
                    save_forensic_failure(
                        timestep=t,
                        omega=omega,
                        Y=res['Y'],
                        dYdt=res['dYdt'],
                        rhs=res['rhs'],
                        align_sup=res['align_sup']
                    )
                    if args.strict:
                        sys.exit(1)

                warnings = float_sanity_check({
                    'Y': res['Y'],
                    'dY/dt': res['dYdt'],
                    'RHS': res['rhs'],
                    'align_sup': res['align_sup']
                })
                for w in warnings:
                    print(w)
                H = compute_entropy(res['_entropic_per_shell'])
                print(f"[entropy] H(t) = {H:.4f}")

            if args.export_cert and (res['dYdt'] is not None) and (res['rhs'] is not None):
                export_certificate(
                    f"/content/output/certificate_{context}.lean",
                    res['dYdt'],
                    res['rhs'],
                    res['C_nl'],
                    res['delta'],
                    res['Lambda'],
                    res['Y'],
                    res['align_sup']
                )

            if args.strict and res['holds'] is False:
                print(f"FAIL: Damping inequality violated at timestep {t}")
                save_forensic_failure(
                    timestep=t,
                    omega=omega,
                    Y=res['Y'],
                    dYdt=res['dYdt'],
                    rhs=res['rhs'],
                    align_sup=res['align_sup']
                )
                sys.exit(1)

            omega = omega_next.copy()

        print(f"Timestep evolution complete for {args.timesteps} steps.")

    else:
        # 5) Single-step certification if timesteps == 0
        res, diagnostics = main_compute(
            omega_t, omega_tpdt,
            grad_u,
            args.alpha,
            args.nu,
            j_min=args.j_min,
            j_max=args.j_max,
            dt=args.dt,
            eps=args.eps,
            plot=args.plot,
            validate=args.run_validation_checks,
            context='mainrun'
        )

        if args.export_cert and (res['dYdt'] is not None) and (res['rhs'] is not None):
            export_certificate(
                "/content/output/certificate.lean",
                res['dYdt'],
                res['rhs'],
                res['C_nl'],
                res['delta'],
                res['Lambda'],
                res['Y'],
                res['align_sup']
            )

        if args.strict and res['holds'] is False:
            print("FAIL: Damping inequality violated for the single-step scenario.")
            save_forensic_failure(
                timestep=0,
                omega=omega_t,
                Y=res['Y'],
                dYdt=res['dYdt'],
                rhs=res['rhs'],
                align_sup=res['align_sup']
            )
            sys.exit(1)

        if args.run_validation_checks:
            try:
                validate_damping_terms(
                    res['Y'], res['dYdt'], res['rhs'],
                    res['align_sup'], res['delta'],
                    res['Lambda'], res['C_nl']
                )
            except AssertionError as e:
                print(f"[VALIDATION ERROR] {e}")
                save_forensic_failure(
                    timestep=0,
                    omega=omega_t,
                    Y=res['Y'],
                    dYdt=res['dYdt'],
                    rhs=res['rhs'],
                    align_sup=res['align_sup']
                )
                if args.strict:
                    sys.exit(1)

            warnings = float_sanity_check({
                'Y': res['Y'],
                'dY/dt': res['dYdt'],
                'RHS': res['rhs'],
                'align_sup': res['align_sup']
            })
            for w in warnings:
                print(w)
            H = compute_entropy(res['_entropic_per_shell'])
            print(f"[entropy] H(t) = {H:.4f}")

        print("Default evolution + certification completed successfully.")
        print("See /content/output/diagnostics_log_mainrun.json for details.")

    # Now run additional diagnostic tests (after the main run)
    if run_blowup:
        simulate_blowup(
            N=args.N, alpha=args.alpha, j_min=args.j_min, j_max=args.j_max,
            dt=args.dt, timesteps=1, strict=args.strict, plot=args.plot,
            run_validation_checks=args.run_validation_checks, nu=args.nu,
            export_cert=args.export_cert, eps=args.eps
        )

    if run_commutator:
        stress_commutator(
            N=args.N, alpha=args.alpha, j_min=args.j_min, j_max=args.j_max,
            dt=args.dt, timesteps=1, strict=args.strict, plot=args.plot,
            run_validation_checks=args.run_validation_checks, nu=args.nu,
            export_cert=args.export_cert, eps=args.eps
        )

    if run_time_reverse:
        time_reverse_test(
            N=args.N, alpha=args.alpha, j_min=args.j_min, j_max=args.j_max,
            dt=args.dt, timesteps=1, strict=args.strict, plot=args.plot,
            run_validation_checks=args.run_validation_checks, nu=args.nu,
            export_cert=args.export_cert, eps=args.eps
        )

    print("All requested runs/tests are complete. Exiting.")

if __name__ == "__main__":
    main()


Overwriting proof_colab.py


In [ ]:
%%writefile recursive_ode_falsifier.py
import glob, json, numpy as np, matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import csv, re, warnings, argparse, sys, os

FALSIFIER_LOG = "/content/output/falsifier_log.txt"

def parse_interval_tuple(s):
    # Accepts: float/int, "X ± Y", [X, Y], (X, Y), or {"mid":X,"width":Y}
    if isinstance(s, (int, float)):
        return (float(s), 0.0)
    if isinstance(s, (list, tuple)) and len(s) == 2 and all(isinstance(x, (int, float)) for x in s):
        return (float(s[0]), float(s[1]))
    if isinstance(s, dict):
        if "mid" in s and "width" in s:
            return (float(s["mid"]), float(s["width"]))
        if "val" in s and "err" in s:
            return (float(s["val"]), float(s["err"]))
        # add more keys as needed
    if isinstance(s, str):
        m = re.match(r"^\s*([-+0-9.eE]+)\s*±\s*([-+0-9.eE]+)\s*$", s)
        if m:
            return (float(m.group(1)), float(m.group(2)))
        try:
            return (float(s), 0.0)
        except:
            pass
    raise ValueError(f"Cannot parse interval from: {s}")


def falsify(reason, strict=False, do_lean_cert=False):
    with open(FALSIFIER_LOG, "a") as f:
        f.write(reason.strip() + "\n")
    print("FALSIFIER:", reason.strip())
    if do_lean_cert:
        export_falsifier_certificate(reason)
    if strict:
        sys.exit(1)

def export_falsifier_certificate(reason, fname="/content/output/falsifier_certificate.lean"):
    with open(fname, "w") as f:
        f.write(f"-- Falsifier certificate: Antagonist claim\n")
        f.write(f"theorem proof_invalid : false := by\n  -- {reason.strip()}\n  contradiction\n")

def compute_entropy(per_shell):
    H = 0.0
    for s in per_shell:
        m, _ = parse_interval_tuple(s['norm'])
        if m > 0:
            H += m * np.log(m)
    return H

def load_logs():
    diagnostics = []
    shell_per_t = []
    files = sorted(glob.glob("/content/output/diagnostics_log_timestep_*.json"),
                  key=lambda f: int(re.search(r'_(\d+)\.json$', f).group(1)))
    if not files:
        raise RuntimeError("No log files found in /content/output/")
    prev_idx = -1
    for f in files:
        with open(f) as j:
            d = json.load(j)
        t_idx = int(re.search(r'_(\d+)\.json$', f).group(1))
        if t_idx <= prev_idx:
            falsify(f"Non-monotonic timestep index in {f}.", strict=False)
        prev_idx = t_idx
        diagnostics.append(d)
        if '_entropic_per_shell' in d:
            shell_per_t.append(d['_entropic_per_shell'])
        else:
            shell_per_t.append([])
    # Guess dt:
    if len(files)>1:
        idxs = [int(re.search(r'_(\d+)\.json$', f).group(1)) for f in files]
        dts = np.diff(idxs)
        dt = float(dts[0]) if np.all(dts==dts[0]) else np.median(dts)
    else:
        dt = 1.0
    return diagnostics, shell_per_t, dt

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--strict", action="store_true", help="Fail hard on any ODE falsification.")
    parser.add_argument("--lean_cert", action="store_true", help="Export Lean-style falsifier certificate.")
    # PATCH: Allow notebook/Colab to pass extraneous args
    args, unknown = parser.parse_known_args()
    diagnostics, shell_per_t, dt = load_logs()
    steps = len(diagnostics)
    t_arr = np.arange(steps) * dt
    # Extract constants and interval data
    Y_tuples  = [parse_interval_tuple(d['Y']) for d in diagnostics]
    Y_mids = [y[0] for y in Y_tuples]
    Y_wids = [y[1] for y in Y_tuples]
    dYdt_vals = [parse_interval_tuple(d['dYdt'])[0] if d['dYdt'] else np.nan for d in diagnostics]
    A_tuples  = [parse_interval_tuple(d['align_sup']) for d in diagnostics]
    A_mids = [a[0] for a in A_tuples]
    A_wids = [a[1] for a in A_tuples]
    C_nl  = float(diagnostics[0]['C_nl'])
    nu    = float(diagnostics[0].get('nu', 0.01))
    delta = float(diagnostics[0]['delta'])
    Lam   = float(diagnostics[0]['Lambda'])
    A_lowers = [max(a[0]-a[1], 0.0) for a in A_tuples]
    A_uppers = [a[0]+a[1] for a in A_tuples]
    Y0_mid = Y_mids[0]; Y0_wid = Y_wids[0]
    # Prepare ODE sim
    def ode_step(Y, A):
        return C_nl * Y**2 * A - nu * Lam * (Y**(1+delta))
    def sim_trajectory(Y0, A_series):
        Ys = [Y0]
        rhses = []
        for i in range(steps-1):
            Y = Ys[-1]
            A = A_series[i]
            rhs = ode_step(Y, A)
            rhses.append(rhs)
            Y_next = Y + dt * rhs
            if Y_next < 0 or np.isnan(Y_next) or np.isinf(Y_next):
                falsify(f"Ode trajectory blew up at t={i*dt:.4g}, Y={Y_next}", strict=args.strict, do_lean_cert=args.lean_cert)
                Y_next = 0.0
            Ys.append(Y_next)
        # Final point for rhs (for completeness)
        rhses.append(ode_step(Ys[-1], A_series[-1]))
        return np.array(Ys), np.array(rhses)
    Ys_sim_nom, rhs_vals_nom = sim_trajectory(Y0_mid, A_mids)
    Ys_sim_lo,  _            = sim_trajectory(max(Y0_mid-Y0_wid,0), A_lowers)
    Ys_sim_hi,  _            = sim_trajectory(Y0_mid+Y0_wid, A_uppers)
    # RHS sign stability check
    if any(r > 0 for r in rhs_vals_nom):
        falsify("RHS > 0 detected: net stretching over damping!", strict=args.strict, do_lean_cert=args.lean_cert)
    # Entropy
    entropies = [compute_entropy(shell) if shell else np.nan for shell in shell_per_t]
    ent_deltas = np.diff(entropies)
    if np.any(ent_deltas > 0.5 * np.abs(entropies[:-1])):
        falsify("Entropy surged >50% between steps — possible intermittent burst", strict=args.strict, do_lean_cert=args.lean_cert)
    # Relative error
    rel_errs = np.abs(Ys_sim_nom - np.array(Y_mids)) / np.maximum(np.abs(Y_mids), 1e-14)
    max_rel_err = np.max(rel_errs)
    trend_measured = Y_mids[-1] < Y_mids[0]
    trend_sim = Ys_sim_nom[-1] < Ys_sim_nom[0]
    trend_match = (trend_measured == trend_sim)
    falsified = False
    violations = []
    for i in range(steps):
        if rel_errs[i] > 0.02:
            msg = f"Step {i}: rel_err={rel_errs[i]:.3%} exceeds Clay threshold (2%)"
            violations.append(msg)
            falsify(f"FALSIFICATION: {msg}", strict=args.strict, do_lean_cert=args.lean_cert)
            falsified = True
        if Ys_sim_nom[i]<0 or np.isnan(Ys_sim_nom[i]):
            msg = f"Step {i}: Simulated Y negative or NaN!"
            violations.append(msg)
            falsify(f"FALSIFICATION: {msg}", strict=args.strict, do_lean_cert=args.lean_cert)
            falsified = True
    if not trend_match:
        falsify("Decay trend between proof logs and ODE simulation does not match (monotonicity fail)", strict=args.strict, do_lean_cert=args.lean_cert)
        falsified = True
    # Derivative check (finite diff)
    dY_log = (np.array(Y_mids[1:]) - np.array(Y_mids[:-1]))/dt
    dY_log_full = np.concatenate([ [dYdt_vals[0]], dY_log ])
    dYdt_agree = np.max(np.abs(dY_log_full-np.array(dYdt_vals))) < 0.01*np.max(np.abs(Y_mids))
    if not dYdt_agree:
        falsify("Log-differs of measured Y(t) and provided dY/dt do not match (inconsistency)", strict=args.strict, do_lean_cert=args.lean_cert)
    # Slope bound check
    if np.any(np.abs(dY_log_full) > 1e3):
        falsify("dY/dt slope magnitude exceeds 1000 — possibly unstable timestep", strict=args.strict, do_lean_cert=args.lean_cert)
    # Plot Y(t)
    plt.figure(figsize=(7,6))
    plt.plot(t_arr, Y_mids, 'o-', label='Measured Y(t)', zorder=3)
    plt.plot(t_arr, Ys_sim_nom, 's-', label='Simulated Y(t) [Nominal]', zorder=2)
    plt.fill_between(t_arr, Ys_sim_lo, Ys_sim_hi, color='lightblue', alpha=0.3, label='Sim interval')
    plt.xlabel('t')
    plt.ylabel('Y(t)')
    plt.title('Recursive ODE (with Antagonist Intervals)')
    plt.legend()
    plt.tight_layout()
    plt.savefig("/content/output/recursive_ode_validation.png")
    # Plot A(t)
    plt.figure(figsize=(7,2))
    plt.plot(t_arr, A_mids, label='A(t) midpoint')
    plt.fill_between(t_arr, A_lowers, A_uppers, color='orange', alpha=0.2, label='A(t) interval')
    plt.ylabel('Alignment A')
    plt.xlabel('t')
    plt.title('Alignment factor interval')
    plt.legend()
    plt.savefig("/content/output/recursive_ode_A.png")
    # Plot entropy
    plt.figure(figsize=(7,2))
    plt.plot(t_arr, entropies, label='Entropy')
    plt.xlabel('t')
    plt.ylabel('H(t)')
    plt.title('Shellwise Entropy')
    plt.legend()
    plt.tight_layout()
    plt.savefig("/content/output/recursive_ode_entropy.png")
    # CSV
    with open("/content/output/recursive_ode_trace.csv", "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['t', 'Y_measured', 'Y_measured_width', 'Y_sim_nom', 'Y_sim_lo', 'Y_sim_hi',
                        'dYdt_measured', 'A_align_mid', 'A_lo', 'A_hi', 'entropy', 'rel_error'])
        for i in range(steps):
            writer.writerow([t_arr[i], Y_mids[i], Y_wids[i], Ys_sim_nom[i],
                            Ys_sim_lo[i], Ys_sim_hi[i], dYdt_vals[i],
                            A_mids[i], A_lowers[i], A_uppers[i], entropies[i], rel_errs[i]])
    print(f"Max relative error: {max_rel_err:.4%}")
    print("Decay trend matches: " if trend_match else "DECAY TREND MISMATCH!")
    print(f"Strict falsified: {falsified}")
    print(f"CSV and plots in /content/output/")

if __name__ == "__main__":
    main()


Overwriting recursive_ode_falsifier.py


In [ ]:
%%writefile proof_falsifier_engine.py
import os
import sys
import shutil
import subprocess
import time
import hashlib
import json
import numpy as np
import zipfile
import datetime
from pathlib import Path

try:
    import cupy as cp
except ImportError:
    cp = None

####### --- Synthetic Field Generators --- #######

def make_shell_localized(j=3, N=32, seed=42):
    # Shell-localized vorticity (Fourier bump at shell j)
    np.random.seed(seed)
    if cp: cp.random.seed(seed)
    omega = np.zeros((3, N, N, N), dtype=np.float64)
    grid = np.fft.fftfreq(N) * N
    Kx, Ky, Kz = np.meshgrid(grid, grid, grid, indexing='ij')
    K2 = Kx**2 + Ky**2 + Kz**2
    mask = (K2 >= 2**(2*j)) & (K2 < 2**(2*(j+1)))
    for c in range(3):
        noise = np.random.randn(N,N,N)
        fft = np.fft.fftn(noise)
        fft *= mask
        loc = np.fft.ifftn(fft).real
        omega[c] = loc / np.sqrt(np.mean(loc**2)+1e-12)
    return omega

def make_vortex_tube(N=32, amp=1.0):
    # Simple vortex tube along e1
    omega = np.zeros((3, N, N, N), dtype=np.float64)
    x = np.linspace(-1,1,N,endpoint=False)
    X,Y,Z = np.meshgrid(x,x,x,indexing='ij')
    r = np.sqrt(Y**2+Z**2)
    profile = amp * np.exp(-20*r**2)
    omega[0] = profile  # Aligned with e1
    return omega

def align_to_e1(omega):
    # Rotate max-energy direction to e1
    v = omega.reshape(3,-1).mean(axis=1)
    if np.linalg.norm(v) < 1e-8:
        return omega
    e1 = np.array([1,0,0.])
    axis = np.cross(v, e1)
    if np.linalg.norm(axis) < 1e-8:
        return omega
    axis = axis / np.linalg.norm(axis)
    theta = np.arccos(np.dot(v, e1)/(np.linalg.norm(v)*np.linalg.norm(e1)))
    from scipy.spatial.transform import Rotation as R
    Rmat = R.from_rotvec(axis*theta).as_matrix()
    return np.tensordot(Rmat, omega, axes=([1],[0]))

def combine_shells(j1=2, j2=6, N=32):
    # Blend two shell-localized fields
    w1 = make_shell_localized(j=j1,N=N,seed=10*j1)
    w2 = make_shell_localized(j=j2,N=N,seed=10*j2+1)
    return 0.5*w1 + 1.5*w2

def random_gaussian(scale=1e-2, N=32, seed=123):
    np.random.seed(seed)
    omega = scale * np.random.randn(3,N,N,N)
    return omega

def proof_evolve(omega_t, dt=1e-4):
    # Very simple Euler step for demonstration (add some noise)
    omega_tpdt = omega_t + dt * np.random.randn(*omega_t.shape) * 0.01
    return omega_tpdt

######## ---- Field Registry ---- ########

fields = [
  {"name": "shell_j3", "generator": make_shell_localized, "params": {"j":3, "seed":42}},
  {"name": "shell_j5", "generator": make_shell_localized, "params": {"j":5, "seed":43}},
  {"name": "aligned_blowup", "generator": make_vortex_tube, "post": "align_to_e1"},
  {"name": "adversarial_combo", "generator": combine_shells, "params": {"j1":2, "j2":6}},
  {"name": "random_noise", "generator": random_gaussian, "params": {"scale":1e-2, "seed":99}},
]

gen_post_map = {
    "align_to_e1": align_to_e1,
}

######## ---- Utility Functions ---- ########
def get_hash(arr):
    return hashlib.md5(arr.astype(np.float32).tobytes()).hexdigest()[:10]

def save_field(arr, fname):
    np.save(fname, arr)

def load_field(fname):
    return np.load(fname)

def get_git_hash():
    try:
        import subprocess
        git_hash = subprocess.check_output(["git", "rev-parse", "HEAD"], cwd="/content/", text=True).strip()
        return git_hash
    except:
        return None

def copy_if_exists(src, dst):
    try:
        if os.path.exists(src):
            shutil.copy(src, dst)
            return True
    except Exception as e:
        print(f"Failed to copy {src} to {dst}: {e}")
    return False

def safe_makedirs(path):
    os.makedirs(path, exist_ok=True)

def bundle_counterexamples(counterexample_paths, zip_out="/content/output/counterexample_bundle.zip"):
    with zipfile.ZipFile(zip_out, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        for cdir in counterexample_paths:
            for fname in Path(cdir).glob('*'):
                zf.write(fname, arcname=f"{Path(cdir).name}/{fname.name}")

######## ---- Metrics & Report ---- ########

summary = {
    "total_runs": 0,
    "proof_colab_failures": 0,
    "ode_falsifier_failures": 0,
    "field_failure_types": [],
    "counterexample_paths": [],
    "entropy_curves": [],
    "run_details": [],
}

######## ---- Main Adversarial Loop ---- ########

def main():
    N = 512
    output_dir = "/content/output/"
    cx_dir = os.path.join(output_dir, "counterexamples/")
    safe_makedirs(output_dir)
    safe_makedirs(cx_dir)
    git_hash = get_git_hash()
    timestamp = datetime.datetime.utcnow().isoformat() + "Z"
    global_run = 1

    for run_idx, field in enumerate(fields, 1):
        name = field['name']
        params = field.get('params', {})
        gen = field['generator']
        context = f"test_{name}_run{run_idx}"
        seed = params.get('seed', 1000+run_idx) if 'seed' in params else 1000+run_idx
        np.random.seed(seed)
        if cp: cp.random.seed(seed)
        run_info = {
            "index": run_idx,
            "context": context,
            "name": name,
            "params": params,
            "timestamp": timestamp,
            "git_hash": git_hash,
            "seed": seed,
        }
        # --- Generate vorticity field ---
        if 'params' in field:
            omega = gen(**params, N=N) if 'N' in gen.__code__.co_varnames else gen(**params)
        else:
            omega = gen(N=N)
        # Apply any postproc:
        if field.get('post'):
            omega = gen_post_map[field['post']](omega)
        omega_t = omega.astype(np.float32)
        save_field(omega_t, '/content/omega_t.npy')
        omega_tpdt = proof_evolve(omega_t)
        save_field(omega_tpdt, '/content/omega_tpdt.npy')
        cx_hash = get_hash(omega_t)
        context_full = f"{context}_{cx_hash}"
        run_info["context_full"] = context_full
        run_info["cx_hash"] = cx_hash
        # Save config:
        config_json = f"/content/output/{context_full}_config.json"
        with open(config_json,"w") as f:
            json.dump(run_info, f, indent=2)
        proof_log = os.path.join(output_dir, 'diagnostics_log.json')
        proof_cert = os.path.join(output_dir, 'proof_certificate.lean')
        falsifier_log = os.path.join(output_dir, 'falsifier_log.txt')
        falsifier_cert = os.path.join(output_dir, 'falsifier_certificate.lean')
        # --- RUN proof_colab.py ---
        proof_args = [
            sys.executable, "proof_colab.py",
            "--input", "omega_t.npy",
            "--input2", "omega_tpdt.npy",
            "--alpha", "2.5", "--plot", "--export_cert",
            "--run_validation_checks", "--strict",
            "--context", context_full
        ]
        summary["total_runs"] += 1
        proc1 = subprocess.run(proof_args, capture_output=True, cwd="/content/")
        proof_failed = proc1.returncode != 0
        ode_failed = False
        # --- RUN recursive_ode_falsifier.py ---
        if not proof_failed:
            ode_args = [sys.executable, "recursive_ode_falsifier.py", "--strict", "--lean_cert"]
            proc2 = subprocess.run(ode_args, capture_output=True, cwd="/content/")
            ode_failed = proc2.returncode != 0
        # --- Handle any failures ---
        failed = False
        cx_this_run_dir = ""
        if proof_failed or ode_failed:
            failed = True
            fail_reason = []
            cxdir = os.path.join(cx_dir, context_full)
            safe_makedirs(cxdir)
            cx_this_run_dir = cxdir
            safe_makedirs(cxdir)
            shutil.copy('/content/omega_t.npy', os.path.join(cxdir, f"{context_full}_omega_t.npy"))
            shutil.copy('/content/omega_tpdt.npy', os.path.join(cxdir, f"{context_full}_omega_tpdt.npy"))
            copy_if_exists(proof_log, os.path.join(cxdir, f"{context_full}_diagnostics_log.json"))
            copy_if_exists(proof_cert, os.path.join(cxdir, f"{context_full}_proof_certificate.lean"))
            copy_if_exists(falsifier_log, os.path.join(cxdir, f"{context_full}_falsifier_log.txt"))
            copy_if_exists(falsifier_cert, os.path.join(cxdir, f"{context_full}_falsifier_certificate.lean"))
            shutil.copy(config_json, os.path.join(cxdir, f"{context_full}_config.json"))
            with open(os.path.join(cxdir, f"{context_full}_proof_stdout.txt"),"w") as f: f.write(proc1.stdout.decode(errors='ignore'))
            with open(os.path.join(cxdir, f"{context_full}_proof_stderr.txt"),"w") as f: f.write(proc1.stderr.decode(errors='ignore'))
            if not proof_failed:
                with open(os.path.join(cxdir, f"{context_full}_ode_stdout.txt"),"w") as f: f.write(proc2.stdout.decode(errors='ignore'))
                with open(os.path.join(cxdir, f"{context_full}_ode_stderr.txt"),"w") as f: f.write(proc2.stderr.decode(errors='ignore'))
            summary["counterexample_paths"].append(cxdir)
            if proof_failed:
                summary["proof_colab_failures"] += 1
                fail_reason.append('proof_colab')
            if ode_failed:
                summary["ode_falsifier_failures"] += 1
                fail_reason.append('recursive_ode_falsifier')
            summary["field_failure_types"].append(f"{name} ({'/'.join(fail_reason)})")
        # Optionally collect entropy curve:
        try:
            diag_path = os.path.join(output_dir, 'diagnostics_log_timestep_000.json')
            if os.path.exists(diag_path):
                with open(diag_path) as f:
                    diag = json.load(f)
                if '_entropic_per_shell' in diag:
                    ent_arr = [float(s.get('norm',0)) for s in diag['_entropic_per_shell']]
                    summary["entropy_curves"].append({"context":context, "entropy": ent_arr})
        except Exception: pass
        # Save run details
        run_info.update({
            "proof_failed": proof_failed,
            "ode_failed": ode_failed,
            "counterexample_dir": cx_this_run_dir
        })
        summary["run_details"].append(run_info)
        result_tag = "[FAIL]" if failed else "[PASS]"
        print(f"{result_tag} {context_full} | proof_colab: {proc1.returncode} | ode: {('n/a' if proof_failed else proc2.returncode)}")
        sys.stdout.flush()

    # Final bundle zip
    if summary["counterexample_paths"]:
        bundle_counterexamples(summary["counterexample_paths"])

    # Print report
    print("------ Adversarial Falsification Test Report ------")
    print(f"Total runs: {summary['total_runs']}")
    print(f"proof_colab.py failures: {summary['proof_colab_failures']}")
    print(f"recursive_ode_falsifier.py failures: {summary['ode_falsifier_failures']}")
    print(f"Field types causing failure:\n  " + "\n  ".join(sorted(set(summary['field_failure_types']))))
    if summary["entropy_curves"]:
        print("Entropy curves available for analysis (first context shown):")
        print(summary["entropy_curves"][0])
    print(f"Counterexamples saved in: {cx_dir}")
    print(f"Bundle zip (if any): /content/output/counterexample_bundle.zip")
    print("--------------------------------------------------")

if __name__ == "__main__":
    main()


Overwriting proof_falsifier_engine.py


In [ ]:
%%writefile certificate_packager.py
#!/usr/bin/env python3

import os
import sys
import json
import glob
import shutil
import hashlib
import argparse
from pathlib import Path
import zipfile
import datetime

############# Utility Functions ################

def sha256file(fn):
    h = hashlib.sha256()
    with open(fn, "rb") as f:
        while True:
            blk = f.read(65536)
            if not blk: break
            h.update(blk)
    return h.hexdigest()

def sha256_bytes(b):
    return hashlib.sha256(b).hexdigest()

def load_json(fn):
    with open(fn,"r") as f:
        return json.load(f)

def fail(msg):
    print(f"[certificate_packager.py] ERROR: {msg}", file=sys.stderr)
    sys.exit(2)

def lean_valid_certificate(lean_path):
    try:
        with open(lean_path,"r") as f:
            data = f.read()
        return "theorem proof_invalid" in data
    except Exception:
        return False

def find_file(prefix, exts, context, extra_match=None):
    """Find file with basename prefix, extension in list, containing context."""
    for e in exts:
        for match in glob.glob(f"**/*{prefix}*{context}*{e}", recursive=True):
            if extra_match and extra_match not in match: continue
            return match
    return None

def find_field_file(ref_dir, hashval, suffix="omega_t.npy"):
    cands = glob.glob(os.path.join(ref_dir, f"**/*{suffix}"), recursive=True)
    for fn in cands:
        try:
            if sha256file(fn) == hashval:
                return fn
        except: pass
    return None

def get_source_hash(path):
    # Take SHA1 of this script as generator source hash
    try:
        with open(path, "rb") as f:
            return hashlib.sha1(f.read()).hexdigest()
    except Exception:
        return "unknown"

def monotonic_decay(yt):
    if not yt or len(yt)<2: return False
    return all((a>=b) for a,b in zip(yt,yt[1:]))

############# Main Capsule Logic ###############

def package_capsules(
    proof_summary_json="/content/output/proof_summary.json",
    output_dir="/content/output/capsules/",
    verify_only=False,
    min_ris=3,
    sign_gpg=False,
):

    # -------------------------------------------------
    # Step 1: Load summary, enumerate valid runs
    # -------------------------------------------------
    if not os.path.exists(proof_summary_json):
        fail(f"proof_summary.json not found at {proof_summary_json}")
    with open(proof_summary_json,"r") as f:
        summary = json.load(f)["summary"] if "summary" in json.load(f) else json.load(f)

    runlist = []
    for run in summary:
        # Must be PASS, have config, lean cert present & valid, relerr ≤2%, monotonic, RIS ≥ min_ris
        if run.get("pass_fail") != "PASS": continue
        if not run.get("lean_cert_present"): continue
        if not run.get("lean_cert_valid"): continue
        if run.get("relerr_max") is None or float(run.get("relerr_max",9e9)) > 0.02: continue
        if not monotonic_decay(run.get("Y_t",[])): continue
        if int(run.get("RIS",0)) < min_ris: continue
        runlist.append(run)
    if not runlist:
        fail("No valid runs found for packaging.")

    Path(output_dir).mkdir(parents=True, exist_ok=True)
    generator_src_hash = get_source_hash(__file__)
    capsules = []
    now = datetime.datetime.utcnow().isoformat()+"Z"

    for run in runlist:
        context = run['context']
        field_type = run.get("type")
        params = run.get("params",{})
        seed = params.get("seed","")
        git_commit = run.get("git_commit","")
        sha_reported = run.get("sha256") or run.get("hash_sha256")
        capsule_data = {
            "context": context,
            "field_type": field_type,
            "seed": seed,
            "params": params,
            "timestamp": now,
            "git_commit": git_commit,
            "sha256": sha_reported,
            "generator_source_hash": generator_src_hash,
            "files": {},
        }
        # ------- Locate and validate all relevant files -------
        # -- omega_t.npy (must match hash from config)
        omega_t_fn = find_field_file("/content/output/", sha_reported, suffix="omega_t.npy")
        if not omega_t_fn:
            fail(f"Run {context}: omega_t.npy with matching hash not found.")
        if sha256file(omega_t_fn) != sha_reported:
            fail(f"Run {context}: hash mismatch for omega_t.npy.")

        # -- omega_tpdt.npy (optional, but must exist if claimed)
        omega_tpdt_fn = None
        for cand in glob.glob(os.path.join(os.path.dirname(omega_t_fn),"*omega_tpdt.npy")):
            if os.path.exists(cand): omega_tpdt_fn = cand

        # -- Config JSON
        config_fn = find_file(prefix="", exts=["_config.json"], context=context)
        if not config_fn:
            fail(f"Run {context}: config json missing.")

        # -- Lean certificate (proof or falsifier), must validate
        lean_fn = find_file(prefix="", exts=[".lean"], context=context)
        if not lean_fn or not lean_valid_certificate(lean_fn):
            fail(f"Run {context}: valid .lean certificate not found or invalid.")

        # -- Diagnostics, logs, preview images, CSVs
        diag_log = find_file(prefix="diagnostics_log", exts=[".json"], context=context)
        csv_fn   = find_file(prefix="", exts=[".csv"], context=context)
        falsifier_log = find_file(prefix="falsifier_log", exts=[".txt"], context=context)
        preview_png = find_file(prefix="", exts=[".png"], context=context)
        # ---- Gather all files to package
        fileset = {
            "omega_t.npy": omega_t_fn,
            "config.json": config_fn,
            "certificate.lean": lean_fn
        }
        if omega_tpdt_fn: fileset["omega_tpdt.npy"] = omega_tpdt_fn
        if diag_log: fileset["diagnostics.json"] = diag_log
        if csv_fn: fileset["summary.csv"] = csv_fn
        if falsifier_log: fileset["falsifier_log.txt"] = falsifier_log
        if preview_png: fileset["preview.png"] = preview_png

        # -- Compute all file hashes
        file_hashes = {name: sha256file(path) for name, path in fileset.items()}
        capsule_data["files"] = file_hashes
        # -- Capsule SHA: hash of (all .npy/.json/.lean data in sorted order)
        cap_bytes = b''
        for k in sorted(fileset.keys()):
            with open(fileset[k],"rb") as f:
                cap_bytes += f.read()
        capsule_sha = sha256_bytes(cap_bytes)
        capsule_data["capsule_sha256"] = capsule_sha

        # -- Archive into ZIP
        capsule_name = f"{context}_capsule.zip"
        capsule_fn = os.path.join(output_dir, capsule_name)
        with zipfile.ZipFile(capsule_fn,"w",compression=zipfile.ZIP_DEFLATED) as zf:
            for fname, src in fileset.items():
                zf.write(src, arcname=fname)
            # manifest will be added after below
        # Save manifest
        manifest_fn = os.path.join(output_dir, f"{context}_capsule_manifest.json")
        with open(manifest_fn, "w") as mf:
            json.dump(capsule_data, mf, indent=2)
        # Add manifest into zip
        with zipfile.ZipFile(capsule_fn,"a",compression=zipfile.ZIP_DEFLATED) as zf:
            zf.write(manifest_fn,"capsule_manifest.json")
        # Optionally: GPG-sign zip (placeholder)
        if sign_gpg:
            # For actual signing, replace this block
            sigtxt = f"signed-by-certificate-packager:{now}"
            sigfile = os.path.join(output_dir, f"{context}_capsule_signature.txt")
            with open(sigfile, "w") as f:
                f.write(sigtxt)
            with zipfile.ZipFile(capsule_fn,"a",compression=zipfile.ZIP_DEFLATED) as zf:
                zf.write(sigfile, "capsule_signature.txt")
        print(f"[OK] Capsule created: {capsule_fn}")
        capsules.append({
            "context": context,
            "capsule_zip": capsule_fn,
            "capsule_manifest": manifest_fn,
            "capsule_sha256": capsule_sha
        })

    # ----------- Summarize ---------------
    print(f"Packaged {len(capsules)} archive capsules into {output_dir}.")
    # Master manifest
    mani = {
        "capsules": capsules,
        "timestamp": now,
        "generator_source_hash": generator_src_hash
    }
    with open(os.path.join(output_dir,"capsule_manifest_master.json"),"w") as f:
        json.dump(mani, f, indent=2)
    print("Master manifest written.")

##########################################

def verify_capsules(capsule_dir="/content/output/capsules/"):
    print(f"Verifying capsules in {capsule_dir} ...")
    manifests = glob.glob(os.path.join(capsule_dir,"*_capsule_manifest.json"))
    zips = glob.glob(os.path.join(capsule_dir,"*_capsule.zip"))
    for mani in manifests:
        with open(mani,"r") as f:
            cap = json.load(f)
        capsule_sha = cap.get("capsule_sha256")
        # find ZIP
        context = cap.get("context")
        zipfile_path = os.path.join(capsule_dir, f"{context}_capsule.zip")
        if not os.path.exists(zipfile_path): fail(f"Capsule zip missing: {zipfile_path}")
        # Check file hashes inside
        with zipfile.ZipFile(zipfile_path,"r") as zf:
            for fname, file_sha in cap["files"].items():
                try:
                    data = zf.read(fname)
                    if sha256_bytes(data) != file_sha:
                        fail(f"[{context}] Hash mismatch for {fname} inside {zipfile_path}")
                except KeyError:
                    fail(f"[{context}] File missing in archive: {fname}")
            # Manifest file:
            mani_data = zf.read("capsule_manifest.json")
            if sha256_bytes(mani_data) != sha256_bytes(json.dumps(cap,sort_keys=True,indent=2).encode()):
                print(f"[WARN] Capsule manifest hash mismatch for {context}")
        # Check Lean cert
        cert_fname = None
        for fname in cap["files"]:
            if fname.endswith(".lean"):
                cert_fname = fname
        if cert_fname:
            with zipfile.ZipFile(zipfile_path,"r") as zf:
                cert_data = zf.read(cert_fname).decode(errors='ignore')
                if "theorem proof_invalid" not in cert_data:
                    fail(f"[{context}] Lean certificate invalid in archive.")
    print("All capsules verified OK.")

##########################################

def main():
    parser = argparse.ArgumentParser(description="Pack and validate proof certificate capsules.")
    parser.add_argument("--summary", type=str, default="/content/output/proof_summary.json")
    parser.add_argument("--output_dir", type=str, default="/content/output/capsules/")
    parser.add_argument("--verify-only", action="store_true")
    parser.add_argument("--sign-gpg", action="store_true")
    parser.add_argument("--min_ris", type=int, default=3)
    args = parser.parse_args()
    if args.verify_only:
        verify_capsules(args.output_dir)
    else:
        package_capsules(
            proof_summary_json=args.summary,
            output_dir=args.output_dir,
            verify_only=False,
            min_ris=args.min_ris,
            sign_gpg=args.sign_gpg
        )

if __name__ == "__main__":
    main()


Overwriting certificate_packager.py


In [ ]:
%%writefile lean_validator.py
#!/usr/bin/env python3

import os
import sys
import re
import csv
import json
import glob
import hashlib
import argparse
import subprocess
from pathlib import Path
from collections import defaultdict
from datetime import datetime

# =================== Helper Functions ===================

def sha256file(fn):
    h = hashlib.sha256()
    with open(fn, "rb") as f:
        for chunk in iter(lambda: f.read(65536), b''):
            h.update(chunk)
    return h.hexdigest()

def md5file(fn):
    h = hashlib.md5()
    with open(fn, "rb") as f:
        for chunk in iter(lambda: f.read(65536), b''):
            h.update(chunk)
    return h.hexdigest()

def get_lean_version():
    try:
        lean_version = subprocess.getoutput("lean --version").strip()
        if lean_version:
            return lean_version
    except Exception:
        pass
    return "Lean unavailable (syntax-only mode)"

def run_lean_check(lean_path):
    try:
        completed = subprocess.run(['lean', '--check', lean_path], capture_output=True, text=True, timeout=15)
        return completed.returncode, completed.stdout, completed.stderr
    except Exception as ex:
        return -99, "", f"EXCEPTION: {ex}"

def parse_theorem_header(contents):
    """
    Extract theorem name, type ("false" or "true"), and statement location.
    """
    header_pattern = re.compile(r'theorem\s+([A-Za-z0-9_]+)\s*:\s*([^:]+):=\s*by', re.S)
    m = header_pattern.search(contents)
    if not m:
        # fallback: more flexible for Lean 4 theorem syntax
        header_pattern2 = re.compile(r'theorem\s+([A-Za-z0-9_]+)\s*:\s*(.*?)\s*:=\s*by', re.S)
        m = header_pattern2.search(contents)
    if m:
        name = m.group(1).strip()
        thm_type = m.group(2).strip()
        is_false = "false" in thm_type
        is_true = "true" in thm_type
        start = m.end()
        return name, (is_false, is_true), start
    return None, (None, None), -1

def extract_proof_body(contents):
    """
    Everything after ':= by'
    """
    sp = contents.split(":= by",1)
    if len(sp)<2:
        return ""
    return sp[1].strip()

def count_proof_steps(body):
    return len([line for line in body.splitlines() if line.strip() and not line.strip().startswith("--")])

def classify_lean_error(stderr):
    if not stderr:
        return None
    if "unknown identifier" in stderr.lower():
        return "Unknown Identifier"
    if "syntax error" in stderr.lower():
        return "Syntax Error"
    if "contradiction" in stderr.lower():
        return "Contradiction"
    if "timeout" in stderr.lower():
        return "Timeout/Tactic Fail"
    if "error" in stderr.lower():
        return "General Error"
    return "Unknown"

def load_json_if_exists(path):
    if not os.path.exists(path): return None
    with open(path,"r") as f:
        return json.load(f)

def smart_str(x):
    if isinstance(x, str): return x
    if isinstance(x, (list, dict)): return json.dumps(x)
    return str(x)

# =================== Main Validation ====================

def main():
    parser = argparse.ArgumentParser(description="Lean certificate validator and proof auditor")
    parser.add_argument("--dir", type=str, required=True, help="Directory with .lean files")
    parser.add_argument("--strict", action="store_true", default=False)
    parser.add_argument("--check_duplicate_ast", action="store_true", default=False)
    args = parser.parse_args()

    root = args.dir
    lean_files = sorted(glob.glob(os.path.join(root, "**/*.lean"), recursive=True))

    lean_version = get_lean_version()
    print(f"Lean version: {lean_version}")

    results = []
    context_proofbody_map = dict()
    ast_hash_map = defaultdict(list)
    duplicates = []
    vacuous = []
    config_mismatches = []
    fail_compiles = []
    lean_hash_set = set()

    for lf in lean_files:
        d = {}
        d['lean_path'] = lf
        d['lean_sha256'] = sha256file(lf)
        d['lean_md5'] = md5file(lf)
        lean_hash_set.add(d['lean_sha256'])
        with open(lf,"r") as f:
            contents = f.read()
        d['lean_size_bytes'] = len(contents.encode())
        theorem_name, (is_false, is_true), thmbody_start = parse_theorem_header(contents)
        d['theorem_name'] = theorem_name
        d['proves_false'] = bool(is_false)
        d['proves_true'] = bool(is_true)
        proof_body = extract_proof_body(contents)
        d['proof_lines'] = count_proof_steps(proof_body)
        d['proof_body_ast_hash'] = hashlib.sha256(proof_body.encode()).hexdigest()
        if args.check_duplicate_ast:
            ast_hash_map[d['proof_body_ast_hash']].append((lf,theorem_name))
        d['lean_version'] = lean_version
        # Trivial/vacuous proof check:
        tauto_match = re.match(r'^\s*(contradiction|trivial)\s*$', proof_body,re.I)
        d['is_trivial'] = bool(tauto_match) or \
            (d['proves_true'] and d['proof_lines']<=1)
        # Associated config
        cfg_path = lf.replace(".lean", "_config.json")
        config = load_json_if_exists(cfg_path)
        d['config_path'] = cfg_path if config else None
        d['has_config'] = bool(config)
        context_hash = config.get("context_full") or config.get("context") if config else None
        d["context_config"] = context_hash
        d['config_field_type'] = config.get("type") if config else "unknown"
        d['config_seed'] = smart_str(config.get("params",{}).get("seed")) if config else ""
        d['config_hash'] = config.get("hash_sha256") or config.get("cx_hash") if config else None
        # RIS scoring
        RIS = 0
        # 1. Compiles under Lean
        rc, out, err = run_lean_check(lf)
        d['lean_compile_returncode'] = rc
        d['lean_stdout'] = out
        d['lean_stderr'] = err
        d['lean_stderr_hash'] = hashlib.sha256((err or "").encode()).hexdigest()
        d['lean_check_status'] = ("PASS" if rc==0 else "FAIL")
        if rc==0:
            RIS += 1
        else:
            fail_compiles.append(lf)
        # 2. Config exists and matches SHA (if possible)
        if config and (
            d['lean_sha256'] == d['config_hash'] or not d['config_hash']
        ):
            RIS += 1
        else:
            config_mismatches.append(lf)
        # 3. Theorem name includes context hash
        if theorem_name and context_hash and context_hash in theorem_name:
            RIS += 1
        else:
            config_mismatches.append(lf)
        # 4. Proof is not trivial/vacuous
        if not d['is_trivial'] and d['proves_false']:
            RIS += 1
        else:
            vacuous.append(lf)
        # 5. Unique proof body
        is_unique = (ast_hash_map[d['proof_body_ast_hash']]==[(lf, theorem_name)]) if args.check_duplicate_ast else True
        if is_unique:
            RIS += 1
        else:
            duplicates.append(lf)
        d['RIS'] = RIS

        # Redundancy check (populate later)
        context_proofbody_map[(context_hash or theorem_name)] = d['proof_body_ast_hash']
        # Collect theorem duplication
        results.append(d)

    # After scan: detect duplicate ASTs (ignore self)
    if args.check_duplicate_ast:
        for proof_hash, lst in ast_hash_map.items():
            if len(lst)>1:
                for lf, thm in lst:
                    for d in results:
                        if d['lean_path']==lf:
                            d['is_duplicate_ast'] = True
                duplicates.extend([x[0] for x in lst if x[0] not in duplicates])

    # ==== Export .csv ====
    summary_csv = os.path.join(root, "lean_validation_summary.csv")
    fieldnames = [
        "lean_path","lean_sha256","lean_md5","theorem_name",
        "proves_false","proves_true","proof_lines","proof_body_ast_hash",
        "lean_version","has_config","context_config","config_field_type",
        "config_seed","config_hash","lean_check_status","RIS"
    ]
    with open(summary_csv,"w",newline="") as f:
        w = csv.DictWriter(f,fieldnames=fieldnames)
        w.writeheader()
        for d in results:
            w.writerow({k:smart_str(d.get(k,"")) for k in fieldnames})

    # ==== Export .md report ====
    summary_md = os.path.join(root, "lean_validation_report.md")
    with open(summary_md,"w") as f:
        f.write(f"# Lean Certificate Validation Report\n")
        f.write(f"Audit time: {datetime.utcnow().isoformat()}Z\n")
        f.write(f"Target directory: `{root}`\n")
        f.write(f"Lean version: {lean_version}\n\n")
        f.write(f"Total certificates: {len(results)}\n\n")
        f.write("## Failures to compile:\n")
        for fn in fail_compiles:
            f.write(f"- {fn}\n")
        f.write("\n## Config mismatches or theorem/context inconsistency:\n")
        for fn in config_mismatches:
            f.write(f"- {fn}\n")
        f.write("\n## Vacuous/trivial certificates:\n")
        for fn in vacuous:
            f.write(f"- {fn}\n")
        if args.check_duplicate_ast:
            f.write("\n## Duplicate AST bodies detected (possible redundancy):\n")
            for fn in duplicates:
                f.write(f"- {fn}\n")
        f.write("\n## RIS Score Histogram\n")
        ris_hist = defaultdict(int)
        for d in results:
            ris_hist[d['RIS']] += 1
        for k in sorted(ris_hist):
            f.write(f"- RIS={k}: {ris_hist[k]}\n")
        f.write("\n## Certificates with RIS < 5\n")
        for d in results:
            if d['RIS']<5:
                f.write(f"- {d['lean_path']} (RIS={d['RIS']})\n")
        f.write("\n\nDetail per certificate is available in the CSV.")

    # ==== Export .json metadata ====
    summary_json = os.path.join(root, "lean_validator_metadata.json")
    for d in results:
        # Truncate large keys to keep JSON readable
        if 'lean_stdout' in d and len(d['lean_stdout'])>500: d['lean_stdout'] = d['lean_stdout'][:500] + "···"
        if 'lean_stderr' in d and len(d['lean_stderr'])>500: d['lean_stderr'] = d['lean_stderr'][:500] + "···"
    with open(summary_json, "w") as f:
        json.dump({"certificates": results, "lean_version": lean_version, "timestamp": datetime.utcnow().isoformat()}, f, indent=2)

    print(f"\nSummary:")
    print(f"- CSV: {summary_csv}")
    print(f"- Markdown report: {summary_md}")
    print(f"- Metadata: {summary_json}")
    if args.strict and (fail_compiles or duplicates or vacuous or config_mismatches):
        print("Strict mode: at least one error detected. Failing.")
        sys.exit(99)
    print("Done.")

if __name__ == "__main__":
    main()


Overwriting lean_validator.py


In [ ]:
%%writefile generate_random_fields.py
#!/usr/bin/env python3
import os
import sys
import hashlib
import json
import time
import socket
import argparse
import numpy as np
from datetime import datetime
from functools import wraps
try:
    import matplotlib.pyplot as plt
except ImportError:
    plt = None

# ================= Registry & Decorators ====================

class FieldRegistry:
    def __init__(self):
        self.generators = {}
    def register(self, name):
        def wrapper(func):
            self.generators[name] = func
            return func
        return wrapper
    def generate(self, name, **params):
        if name not in self.generators:
            raise ValueError(f"Unknown field type: {name}")
        return self.generators[name](**params)

field_registry = FieldRegistry()

def validate_field(fn):
    """Decorator for generator functions to check output shape, finiteness, divergence."""
    @wraps(fn)
    def wrapper(*args, **kwargs):
        omega = fn(*args, **kwargs)
        if omega.shape[0] != 3 or len(omega.shape) != 4:
            raise ValueError(f"Field shape invalid: expected (3,N,N,N), got {omega.shape}")
        if not np.isfinite(omega).all():
            raise ValueError("Field contains inf/nan values")
        if np.linalg.norm(omega) < 1e-8:
            raise ValueError("Field norm too small")
        # Divergence-free test (optional unless forced)
        if kwargs.get('enforce_divergence', True):
            N = omega.shape[1]
            dx = 1.0/N
            div = (
                np.gradient(omega[0], dx, axis=0) +
                np.gradient(omega[1], dx, axis=1) +
                np.gradient(omega[2], dx, axis=2)
            )
            if np.abs(div).mean() >= 1e-3:
                raise ValueError("Field failed divergence test: mean(abs(div)) >= 1e-3")
        return omega
    return wrapper

# ============= Field Generators =========================

@field_registry.register("shell_random")
@validate_field
def shell_random(j=8, N=512, seed=42, amplitude=1.0, **_):
    """Divergence-free field with energy in shell |k| ~ 2^j."""
    np.random.seed(seed)
    F = np.zeros((3, N, N, N), dtype=np.complex64)
    freq = np.fft.fftfreq(N)*N
    Kx, Ky, Kz = np.meshgrid(freq, freq, freq, indexing='ij')
    K2 = Kx**2 + Ky**2 + Kz**2
    mask = (K2 >= 2**(2*j)) & (K2 < 2**(2*(j+1)))
    if not np.any(mask):
        max_freq = np.max(np.abs(freq))
        j_adj = int(np.floor(np.log2(max_freq)))
        mask = (K2 >= 2**(2*j_adj)) & (K2 < 2**(2*(j_adj+1)))
    idx = np.nonzero(mask)
    if idx[0].size > 0:
        kvec = np.vstack([Kx[idx], Ky[idx], Kz[idx]])  # shape (3, M)
        norm_k = np.linalg.norm(kvec, axis=0)
        valid = norm_k >= 1e-7
        if np.any(valid):
            kvec_valid = kvec[:, valid]
            norm_k_valid = norm_k[valid]
            k_unit = kvec_valid / norm_k_valid  # shape (3, M_valid)
            M_valid = k_unit.shape[1]
            u = np.random.randn(3, M_valid) + 1j * np.random.randn(3, M_valid)
            dot = np.sum(u * k_unit, axis=0)
            u = u - k_unit * dot
            indices = np.array(np.nonzero(mask)).T  # shape (M, 3)
            indices_valid = indices[valid]
            F[:, indices_valid[:,0], indices_valid[:,1], indices_valid[:,2]] = u
    norm_F = np.sqrt((np.abs(F)**2).sum())
    if norm_F == 0:
        raise ValueError("No energy in selected Fourier modes; adjust 'j' or 'N'.")
    F /= norm_F
    F *= amplitude
    omega = np.fft.ifftn(F, axes=(1,2,3)).real.astype(np.float32)
    return omega

@field_registry.register("multi_shell")
@validate_field
def multi_shell(j1=6, j2=9, N=512, seed=888, amplitude=1.0, **_):
    """Superpose two shells."""
    a = shell_random(j=j1, N=N, seed=seed, amplitude=amplitude/2, enforce_divergence=False)
    b = shell_random(j=j2, N=N, seed=seed+1, amplitude=amplitude/2, enforce_divergence=False)
    omega = a + b
    return omega

@field_registry.register("aligned_noise")
@validate_field
def aligned_noise(N=512, seed=123, amplitude=1.0, **_):
    np.random.seed(seed)
    omega = np.random.randn(3, N, N, N)
    v = omega.reshape(3,-1).mean(axis=1)
    if np.linalg.norm(v) > 1e-8:
        theta = np.arccos(np.dot(v, [1,0,0])/np.linalg.norm(v))
        if theta > 1e-5:
            axis = np.cross(v, [1,0,0])
            axis = axis/np.linalg.norm(axis)
            from scipy.spatial.transform import Rotation as R
            rot = R.from_rotvec(axis*theta)
            omega = np.tensordot(rot.as_matrix(), omega, axes=([1],[0]))
    omega *= amplitude / np.sqrt(np.mean(omega**2)+1e-12)
    return omega.astype(np.float32)

@field_registry.register("white_noise")
@validate_field
def white_noise(N=512, seed=77, amplitude=1.0, **_):
    np.random.seed(seed)
    omega = amplitude * np.random.randn(3, N, N, N).astype(np.float32)
    return omega

@field_registry.register("vortex_tube")
@validate_field
def vortex_tube(N=512, amplitude=1.0, **_):
    x = np.linspace(-1,1,N,endpoint=False)
    X,Y,Z = np.meshgrid(x,x,x,indexing='ij')
    r = np.sqrt(Y**2+Z**2)
    tube = amplitude * np.exp(-30*r**2)
    omega = np.zeros((3,N,N,N), dtype=np.float32)
    omega[0] = tube
    return omega

@field_registry.register("boundary_layer")
@validate_field
def boundary_layer(N=512, amplitude=1.0, **_):
    x = np.linspace(-1,1,N,endpoint=False)
    Z = np.meshgrid(x,x,x,indexing='ij')[2]
    layer = amplitude * np.exp(-200*Z**2)
    omega = np.zeros((3,N,N,N), dtype=np.float32)
    omega[1] = layer
    return omega

@field_registry.register("adversarial_mix")
@validate_field
def adversarial_mix(N=512, j=3, j2=6, seed=444, amplitude=1.0, **_):
    field1 = shell_random(j=j, N=N, seed=seed, amplitude=amplitude/2, enforce_divergence=False)
    field2 = white_noise(N=N, seed=seed+993, amplitude=amplitude/4, enforce_divergence=False)
    field3 = vortex_tube(N=N, amplitude=amplitude/4, enforce_divergence=False)
    omega = field1 + field2 + field3
    return omega

# ============ Evolution Stepper =======================

def proof_evolve(omega_t, dt=1e-4, magnitude=0.01, seed=2023):
    np.random.seed(hash(float(np.sum(omega_t))+dt+seed) % (2**32-1))
    noise = magnitude * np.random.randn(*omega_t.shape)
    return omega_t + dt*noise

# ========== Hashing & Metadata ========================

def get_sha256(arr):
    m = hashlib.sha256()
    m.update(arr.tobytes())
    return m.hexdigest()

def get_md5(arr):
    m = hashlib.md5()
    m.update(arr.astype(np.float32).tobytes())
    return m.hexdigest()

def get_src_hash(fn):
    co = fn.__code__
    code_bytes = co.co_code
    h = hashlib.sha1(code_bytes).hexdigest()
    return h

def get_git_commit():
    try:
        import subprocess
        commit = subprocess.check_output(['git', 'rev-parse', 'HEAD'], text=True).strip()
        return commit
    except Exception:
        return "unknown"

# ========== CLI & Main ===============================

def visualize_field(omega, out_fn="preview_slice.png"):
    import matplotlib.pyplot as plt
    N = omega.shape[1]
    mid = N//2
    fig, axs = plt.subplots(1, 3, figsize=(10, 3))
    for i, comp in enumerate(['x', 'y', 'z']):
        im = axs[i].imshow(omega[i, mid], cmap='RdBu', vmax=np.abs(omega[i, mid]).max())
        axs[i].set_title(f"$\\omega_{comp}$ (z={mid})")
        plt.colorbar(im, ax=axs[i])
    plt.tight_layout()
    plt.savefig(out_fn)
    plt.close(fig)

def main():
    parser = argparse.ArgumentParser(description="Generate reproducible random vorticity fields for proof engine.")
    parser.add_argument('--type', type=str, required=True, help="Field type")
    parser.add_argument('--j', type=int, help="Shell/Multi")
    parser.add_argument('--j1', type=int)
    parser.add_argument('--j2', type=int)
    parser.add_argument('--seed', type=int, default=42)
    parser.add_argument('--amplitude', type=float, default=1.0)
    parser.add_argument('--N', type=int, default=512)
    parser.add_argument('--out', type=str, default="/content/omega_t.npy")
    parser.add_argument('--visualize', action='store_true')
    parser.add_argument('--evolve', action='store_true')
    parser.add_argument('--export_config', action='store_true')
    args = parser.parse_args()

    params = {}
    argsvars = vars(args)
    for k in argsvars:
        v = argsvars[k]
        if k in {'visualize', 'evolve', 'out', 'export_config', 'type'} or v is None:
            continue
        params[k] = v

    ftype = args.type
    if ftype not in field_registry.generators:
        raise ValueError(f'Field type not recognized: {ftype}')
    fn = field_registry.generators[ftype]
    import inspect
    valid_args = inspect.getfullargspec(fn).args
    fn_params = {k: params[k] for k in params if k in valid_args}

    omega = field_registry.generate(ftype, **fn_params)
    assert omega.shape == (3, args.N, args.N, args.N)
    save_path = args.out
    np.save(save_path, omega)

    sha256 = get_sha256(omega)
    md5 = get_md5(omega)
    src_hash = get_src_hash(fn)
    git_commit = get_git_commit()
    ts = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
    hostname = socket.gethostname()

    if args.visualize and plt is not None:
        out_img = os.path.splitext(save_path)[0] + "_preview.png"
        visualize_field(omega, out_img)

    if args.evolve:
        omega_tpdt = proof_evolve(omega)
        out_tpdt = os.path.splitext(save_path)[0] + "_tpdt.npy"
        np.save(out_tpdt, omega_tpdt)

    if args.export_config:
        config = {
            "type": ftype,
            "params": fn_params,
            "shape": list(omega.shape),
            "hash_sha256": sha256,
            "float_hash_md5": md5,
            "generator_source_hash": src_hash,
            "timestamp": ts,
            "git_commit": git_commit,
            "hostname": hostname
        }
        config_path = os.path.splitext(save_path)[0] + "_config.json"
        with open(config_path, "w") as f:
            json.dump(config, f, indent=2)
        print(f"Config saved to {config_path}")

    print(f"Field '{ftype}' written to {save_path}")
    print(f"SHA256: {sha256}")
    print(f"MD5(float32): {md5}")

if __name__ == "__main__":
    main()


Overwriting generate_random_fields.py


In [ ]:
# Step 1: Generate a field with real energy in j=8 shell
!python3 generate_random_fields.py --type multi_shell --j1 6 --j2 9 --N 512 --seed 123 --evolve --visualize --export_config

fatal: not a git repository (or any of the parent directories): .git
Config saved to /content/omega_t_config.json
Field 'multi_shell' written to /content/omega_t.npy
SHA256: 7d09439c791610e3f39c3b6ebac02e1880f15f22aeabffb315916be8980dc4f4
MD5(float32): 635653f02ace9faa614018ead6c5b45a


In [ ]:
!python proof_colab.py --input omega_t.npy --input2 omega_t_tpdt.npy --alpha 2.5 --j_min 6 --j_max 9 --timesteps 100 --strict --export_cert --plot --run_validation_checks


Using CuPy (GPU) for arrays and FFTs
Loaded omega_t from omega_t.npy, shape (3, 512, 512, 512).
Loaded omega_tpdt from omega_t_tpdt.npy, shape (3, 512, 512, 512).
Running 100 time steps with dt=0.0001 and verifying at each step.
  (Computing velocity and grad_u from vorticity via Biot-Savart)
[biot_savart] max|div u| = 5.02e-15

Recursive norm Y(t):          0.0103605 ± 1.04e-09
sup_j alignment:              0.500082 ± 5.00e-08
dY/dt:                        -11.7085 ± 1.96e-05
RHS:                          0.0713894 ± 2.14e-08
[Inequality verified]: This timestep is within bounds.

[entropy] H(t) = -0.0000
Wrote Lean-style proof certificate to /content/output/certificate_timestep_000.lean
  (Computing velocity and grad_u from vorticity via Biot-Savart)
[biot_savart] max|div u| = 4.57e-15

Recursive norm Y(t):          0.00939181 ± 9.39e-10
sup_j alignment:              0.500079 ± 5.00e-08
dY/dt:                        -10.1023 ± 1.78e-05
RHS:                          0.0586632 ± 1.76e-

In [ ]:
!python3 proof_colab.py --run_blowup_test --strict
!python3 proof_colab.py --run_commutator_test --strict
!python3 proof_colab.py --run_time_reverse_test --strict

Using CuPy (GPU) for arrays and FFTs
No --input provided. Generating a synthetic 'vortex tube' field.
  (Computing velocity and grad_u from vorticity via Biot-Savart)
[biot_savart] max|div u| = 0.00e+00

Recursive norm Y(t):          347872 ± 3.48e-02
sup_j alignment:              0.411426 ± 4.11e-08
dY/dt not available (only single time). No inequality check.
Default evolution + certification completed successfully.
See /content/output/diagnostics_log_mainrun.json for details.
[blowup test] j_min=6, j_max=9, timesteps=1
  (Computing velocity and grad_u from vorticity via Biot-Savart)
[biot_savart] max|div u| = 9.26e-05

Recursive norm Y(t):          2.48765 ± 2.49e-07
sup_j alignment:              0.329752 ± 3.30e-08
dY/dt:                        -4629.68 ± 4.51e-03
RHS:                          2714.19 ± 8.14e-04
[Inequality verified]: This timestep is within bounds.

[blowup test] Completed.
All requested runs/tests are complete. Exiting.
Using CuPy (GPU) for arrays and FFTs
No --in

In [ ]:
!python3 recursive_ode_falsifier.py --strict --lean_cert

Traceback (most recent call last):
  File "/content/recursive_ode_falsifier.py", line 211, in <module>
    main()
  File "/content/recursive_ode_falsifier.py", line 125, in main
    Ys_sim_nom, rhs_vals_nom = sim_trajectory(Y0_mid, A_mids)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/recursive_ode_falsifier.py", line 115, in sim_trajectory
    rhs = ode_step(Y, A)
          ^^^^^^^^^^^^^^
  File "/content/recursive_ode_falsifier.py", line 108, in ode_step
    return C_nl * Y**2 * A - nu * Lam * (Y**(1+delta))
                  ~^^~
OverflowError: (34, 'Numerical result out of range')


In [ ]:
!python3 proof_falsifier_engine.py

fatal: not a git repository (or any of the parent directories): .git
[FAIL] test_shell_j3_run1_ec23a7ff2c | proof_colab: 2 | ode: n/a
[FAIL] test_shell_j5_run2_47a2d52362 | proof_colab: 2 | ode: n/a
[FAIL] test_aligned_blowup_run3_296c65368f | proof_colab: 2 | ode: n/a
[FAIL] test_adversarial_combo_run4_4fae59cf96 | proof_colab: 2 | ode: n/a
[FAIL] test_random_noise_run5_cda5c21d99 | proof_colab: 2 | ode: n/a
------ Adversarial Falsification Test Report ------
Total runs: 5
proof_colab.py failures: 5
recursive_ode_falsifier.py failures: 0
Field types causing failure:
  adversarial_combo (proof_colab)
  aligned_blowup (proof_colab)
  random_noise (proof_colab)
  shell_j3 (proof_colab)
  shell_j5 (proof_colab)
Counterexamples saved in: /content/output/counterexamples/
Bundle zip (if any): /content/output/counterexample_bundle.zip
--------------------------------------------------


In [ ]:
!python3 certificate_packager.py

[certificate_packager.py] ERROR: proof_summary.json not found at /content/output/proof_summary.json


In [ ]:
!python3 lean_validator.py --dir /content/output/counterexamples/ --strict --check_duplicate_ast

Lean version: /bin/sh: 1: lean: not found

Summary:
- CSV: /content/output/counterexamples/lean_validation_summary.csv
- Markdown report: /content/output/counterexamples/lean_validation_report.md
- Metadata: /content/output/counterexamples/lean_validator_metadata.json
Done.


In [ ]:
!python3 lean_validator.py --dir /content/output/ --strict --check_duplicate_ast

Lean version: /bin/sh: 1: lean: not found

Summary:
- CSV: /content/output/lean_validation_summary.csv
- Markdown report: /content/output/lean_validation_report.md
- Metadata: /content/output/lean_validator_metadata.json
Strict mode: at least one error detected. Failing.


In [ ]:
!zip -r navier_stokes_proof_output.zip /content/output/

  adding: content/output/ (stored 0%)
  adding: content/output/inequality_timestep_081.png (deflated 20%)
  adding: content/output/diagnostics_log_timestep_094.json (deflated 67%)
  adding: content/output/certificate_timestep_051.lean (deflated 31%)
  adding: content/output/inequality_timestep_093.png (deflated 20%)
  adding: content/output/certificate_timestep_099.lean (deflated 31%)
  adding: content/output/inequality_timestep_008.png (deflated 20%)
  adding: content/output/per_shell_timestep_028.png (deflated 23%)
  adding: content/output/diagnostics_log_timestep_081.json (deflated 67%)
  adding: content/output/diagnostics_log_timestep_044.json (deflated 67%)
  adding: content/output/certificate_timestep_048.lean (deflated 32%)
  adding: content/output/diagnostics_log_timestep_051.json (deflated 67%)
  adding: content/output/diagnostics_log_timestep_033.json (deflated 67%)
  adding: content/output/inequality_timestep_015.png (deflated 21%)
  adding: content/output/certificate_timest

In [ ]:
!zip -r navier_stokes_proof_full_output.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.03.05/ (stored 0%)
  adding: content/.config/logs/2025.03.05/14.26.32.612701.log (deflated 58%)
  adding: content/.config/logs/2025.03.05/14.26.41.012425.log (deflated 57%)
  adding: content/.config/logs/2025.03.05/14.26.30.965194.log (deflated 86%)
  adding: content/.confi